# Descrição do Problema

Na investigação de fraudes em licitações, existem muitos desafios. Diferentes ações fraudulentas, pouco diferentes das não fraudulentas. Informações descentralizada entre os dados abertos governamentais e dados abertos do setor privado.  

# Objetivos

* investigar dados de licitações do GDF relacionados a prestação de serviço de TIC entre os anos de 2014 e 2020

* Identificar empresas inedonias que continuam disputando licitações
* Identificar empresas diferentes com mesmo corpo diretor
* Investigar contratos com dispensa de licitação realizados com essas empresas


# Recursos


- Trello
- Google Colab
- Python

In [2]:
# Bibliotecas Py

import pandas as pd
import requests
import json
import numpy as np
import itertools
import shutil
from google.colab import drive
from google.colab import files

# drive.mount('drive')

ModuleNotFoundError: No module named 'pandas'

# Fontes de dados

Fontes de dados utilizadas:

* SIASG - Sistema Integrado de Administração de Serviços Gerais, Ministério da Economia
 
* CEIS - Cadastro Nacional de Empresas Inidôneas e Suspensas, Controladoria Gera da União - CGU

* CNEP - Cadastro Nacional de Empresas Punidas,

  - Portal de Compras Governamentais
    - [API](http://compras.dados.gov.br/)
    - [Dicionário de dados](http://compras.dados.gov.br/docs/lista-metodos-licitacoes.html)

  - Portal da Transparência
    - [API](http://api.portaldatransparencia.gov.br/)
    - [Documentação](#)
    - [Dicionário de dados](http://www.portaldatransparencia.gov.br/pagina-interna/603389-dicionario-de-dados-licitacoes)


* Projeto de compartilhamento de dados do ICIJ - [Datashare](https://datashare.icij.org/) 

#### ORGÃOS DO DF > UASG > LICITACOES > DESPESAS > FORNECEDORES

# Descricão dos dados - Transparência

### Órgãos do DF

### Despesas por órgão

In [ ]:
chave = {'chave-api-dados':'b07b2db3a9698e2a350ef9d36d0eb280'}

In [ ]:
json = []

for ano in range(2015, 2020):
  for i in orgaos:
    url= f'http://api.portaldatransparencia.gov.br/api-de-dados/despesas/por-orgao?ano={ano}&orgao={i}&pagina=1'
    # print(url)
    res = requests.get(url, verify=True,headers=chave).json()
    json = json + res
  
json = list(filter(None, json))
dfOrgaosDespesas = pd.DataFrame(json) 

In [ ]:
json[0]

In [ ]:
dfOrgaosDespesas.head()

# Descrição dos dados - compras.gov.br

ORGÃOS DO DF > UASG > LICITACOES > CONTRATOS > MODALIDADES 

PREGAO > RESULTADO > FORNECEDOR > ITENS > VALORES LANCES > PROPOSTAS > EVENTOS


### Órgãos do DF

#### metadados


- ativo	- Se o órgão está ativo.
- codigo - Código do órgão
- codigo_tipo_adm	- Código do tipo da administração do órgão
- codigo_tipo_esfera	- Tipo da esfera do órgão
- codigo_tipo_poder	- Código do tipo do poder do órgão
- nome	- Nome do órgão.

http://compras.dados.gov.br/licitacoes/doc/orgao/54000

#### execução

In [ ]:
# download API

res = requests.get('http://compras.dados.gov.br/licitacoes/v1/orgaos.json?nome=Distrito+federal', verify=True).json()
jsonOrgaos = res['_embedded']['Orgaos']
dfOrgao = pd.DataFrame(jsonOrgaos)

In [ ]:
# leitura arquivo one drive

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

In [ ]:
dfOrgao[['codigo','nome','codigo_tipo_esfera','ativo']].head()

In [ ]:
dfOrgao.to_csv('/content/drive/My Drive/Colab Notebooks/TCC/dados_fonte/orgaos.csv', mode='w', header=True)

### UASGs relacionadas e Órgãos Superiores

#### metadados

UASG: Unidade Administrativa de Serviços Gerais do SIASG

- ativo -	Se a UASG está ativa.
- cep -	CEP da UASG.
- cnpj -	CNPJ da UASG
- id -	Identificador único da UASG no SICAF.
- id_municipio -	Identificador único do município da UASG.
- id_orgao -	Identificador único do órgão no SICAF.
- id_orgao_superior -	Identificador único do órgão Superior.
- nome -	Nome da UASG.
- total_fornecedores_cadastrados -	Quantidade total de fornecedores cadastrados na UASG.
- total_fornecedores_recadastrados -	Quantidade total de fornecedores recadastrados, no novo SICAF, na UASG.
- unidade_cadastradora -	Unidade cadastradora.

http://compras.dados.gov.br/docs/licitacoes/v1/uasgs.html

#### execucao

In [ ]:
arrUasg = []

for i in dfOrgao.codigo:
  # print(str(i))
  res = requests.get('http://compras.dados.gov.br/licitacoes/v1/uasgs.json?id_orgao='+str(i), verify=True).json()
  arrUasg = arrUasg + res['_embedded']['uasgs']

dfUasg = pd.DataFrame(arrUasg) 
dfUasg.columns


In [ ]:
dfUasg.to_csv('/content/drive/My Drive/Colab Notebooks/TCC/dados_fonte/uasgs.csv', mode='w', header=True)

In [ ]:
orgaos = dfUasg.id_orgao.unique()
orgaos_super = dfUasg.id_orgao_superior.unique()
print(orgaos_super)

### Licitações por UASG

### Contratos vinculados as licitacões

#### metadados

- codigo_do_item_no_catalogo -	Código do Item no Catálogo.
- data_abertura_proposta -	Data de abertura da proposta.
- data_entrega_edital -	Data de Entrega do Edital.
- data_entrega_proposta -	Data de entrega da proposta.
- data_publicacao -	Data da publicação da licitação.
- endereco_entrega_edital -	Endereço de Entrega do Edital.
- funcao_responsavel -	Função do Responsável pela Licitação.
- identificador -	Identificador da Licitação.
- informacoes_gerais -	Informações Gerais.
- modalidade -	Código da Modalidade da Licitação.
- nome_responsavel -	Nome do Responsável pela Licitação.
- numero_aviso -	Número do Aviso da Licitação.
- numero_item_licitacao -	Número Item Licitação.
- numero_itens -	Número de Itens.
- numero_processo -	Número do Processo.
- objeto -	Objeto da Licitação.
- situacao_aviso -	Situação do aviso.
- tipo_pregao -	Tipo do Pregão.
- tipo_recurso -	Tipo do Recurso.
- uasg -	Código da UASG.

http://compras.dados.gov.br/docs/licitacoes/v1/licitacoes.html 

#### execucao

In [ ]:
dfUasg.head()

In [ ]:
# exemplo retorno

# http://compras.dados.gov.br/licitacoes/v1/licitacoes.json?uasg=389144
# print(url)
res = requests.get(f'{api}', verify=True, stream=True).json()

print(res)
# paginação

In [ ]:
json = []
offset = 0
session = requests.Session()

for i in dfUasg.id:
  url = f'http://compras.dados.gov.br/licitacoes/v1/licitacoes.json?uasg={i}&offset={offset}'
  print(url)
  offset = offset + 500
  with requests.get(f'{url}&offset={offset}', verify=True, stream=True).json() as r:
    r = session.get(url, stream = True, verify=True).json()
    r.raise_for_status()
    for chunk in r.iter_content(500):
        if(r['_embedded']['licitacoes'] != 0):
          json.append(r['_embedded']['licitacoes'])

dfLicitacoes = pd.DataFrame(json)

In [ ]:
# teste = pd.DataFrame() (json['_embedded']['licitacoes'])


In [ ]:

json['_embedded']['licitacoes']



In [ ]:
# http://compras.dados.gov.br/contratos/v1/contratos.html?uasg=20001&order_by=data_assinatura&order=desc

### Modalidades de licitações

In [ ]:
url = 'http://compras.dados.gov.br/licitacoes/v1/modalidades_licitacao.json'
json = requests.get(url, verify=True).json()

In [ ]:
dfModalidades = pd.DataFrame(json['_embedded']['ModalidadesLicitacao'])

In [ ]:
pd.set_option('display.max_colwidth', None)
dfModalidades[['codigo','descricao']]

### Grupos relacionados a TIC

In [ ]:
json = requests.get('http://compras.dados.gov.br/servicos/v1/grupos.json?descricao=TIC', verify=True).json()

obj = json['_embedded']['grupos']

df = pd.DataFrame(obj)
dfGrupos = df[['codigo','descricao']]
dfGrupos = dfGrupos[dfGrupos.codigo.isin([162,163,164,165,166,168,171,172,173,174,800])]

In [ ]:
dfGrupos

### Despesas por Órgão - compras.dados.gov